## First lets setup some configuration variables for the notebook

In [8]:
# We use Poetry to setup the python environment and dependancies
# https://python-poetry.org/docs/basic-usage/

# Run the following command to install the dependancies
# poetry install


# Define the path for the data file (csv) as exported from Survey Monkey
data_folder = '../survey_data/'

csv_survey_file = data_folder + 'surveydata.csv'  # Replace with desired input file path
csv_survey_merged_output = data_folder + 'survey_data_merged.csv'  # Replace with desired output file path

#import os
#print(os.getcwd())


## 1. Assuming you downloaded the CSV from SurveyMonkey lets prep the data

### 1.1. First we merge the column options into a single column with a semicolon separator for each option

| Name | Favorite color |  |  | Age | Food Likes |  |  |
| --- | --- | --- | --- | --- | --- | --- | --- |
|  | Blue | Red | Yellow |  | Meat | Veg | Fish |
| John |  | Red |  | 30 |  | Veg | Fish |
| Mary | Blue |  |  | 20 | Meat |  |  |
| June |  |  | Yellow | 43 | Meat | Veg |

We want to have this:

| Name | Favorite color | Age | Food Likes |
| --- | --- | --- | --- |
| John | Red | 30 | Veg; Fish |
| Mary | Blue | 20 | Meat |
| June | Yellow | 43 | Meat; Veg |

In [ ]:
import pandas as pd

def process_csv(file_path, output_file_path):
    df = pd.read_csv(file_path, header=None)

    # Headers
    headers = df.iloc[0]

    # Identify start and end indices of each group of option columns
    option_indices = []
    start_index = None
    for i, header in enumerate(headers):
        if pd.isna(header):
            if start_index is None:
                start_index = i - 1
        elif start_index is not None:
            option_indices.append((start_index, i))
            start_index = None
    if start_index is not None:
        option_indices.append((start_index, len(headers)))

    # Process data rows
    processed_data = []
    for i in range(2, len(df)):
        row = df.iloc[i]
        new_row = []
        last_index = 0
        for start, end in option_indices:
            # Add non-option columns as they are
            new_row.extend(row[last_index:start])
            # Merge option columns
            values = row[start:end].dropna()
            concatenated_values = '; '.join(values.astype(str))
            new_row.append(concatenated_values)
            last_index = end

        # Add remaining columns if any
        if last_index < len(headers):
            new_row.extend(row[last_index:])

        processed_data.append(new_row)

    # Create a new DataFrame for output
    new_headers = [header for header in headers if pd.notna(header)]
    output_df = pd.DataFrame(processed_data, columns=new_headers)
    output_df.to_csv(output_file_path, index=False)
    return output_df


# Process the CSV
output_df = process_csv(csv_survey_file, csv_survey_merged_output)
print(output_df.head())


### 1.2. The respondant IDs can have issues from CSV export with Survey Monkey
So we wan tto use the row number as the ID so replace the first column with the row number

In [ ]:
# Replace the first column values with the row number isntead of the Survey Monkey repondant ID which exports as a number in scientific notation
output_df['Respondent ID'] = output_df.index + 1
print(output_df.head())
# overwrite the output file
output_df.to_csv(csv_survey_merged_output, index=False)


### 1.3 Now lets work on the data header columns, using AI to make the long questions into "codes" or short titles

### This section uses AI, in paritcular we are using OpenAI's GPT-3.5 latest model
The cost is (as of Jan 2024), $0.0001 per 1000 tokens which means if a prompt has say 50 words the data is 100 words, and and prompt response is 100 words, then thats 250 words (350 tokens). Say there are 200 survey entries, thats 350x200 = 70,000 tokens. So thats $0.007 to run a batch of 200 surveys.

You will need the API key in the `.env` file as `OPENAI_API_KEY`

We also use `langchain` to do all the LLM interactions, its just easier.
It will get installed with the initil `poetry install` command.

https://python.langchain.com/docs/get_started


In [3]:
# Define the survey data file we want to use
csv_survey = data_folder + 'survey_data_merged.csv'  # Replace with desired output file path

# The AI language model
model = "gpt-3.5-turbo-1106"

# get the API key from the environment variable
import os
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

chat = ChatOpenAI(model=model)


### Creating shorter question titles

Lets dump all the questions, one per line, to a language model and have it come up with shorter titles for each column.
making sure they are all unique. Then return the full data with better titles.
We will use CamelCase for the titles, with max 5 words per title.

This can take 10 seconds as the Language model writes it all out

We will end up with a new CSV file where the column titls are shorter and more readable.
We will also create a map for the short titles to the long titles as we might need the long titles (questions) later for the language model to understand them.

In [5]:
# Lets dump all the questions, one per line, to a language model and have it come up with shorter titles for each column.
# making sure they are all unique. Then return the full data with better titles.
# We will use CamelCase for the titles, with max 5 words per title.


# define the prompt
tpl = """ 
You will be given the questions from a survey, one per line and you will need to come up with a shorter title for each question.
Use CamelCase for the titles, with max 5 words per title.
Only provide the new titles, one per line, in the same order as the questions.

Questions:
{questions}
"""

# get the questions from the df from the survey data
import pandas as pd
df = pd.read_csv(csv_survey)
questions = df.columns.values.tolist()
# list as one per line
questions = "\n".join(questions)
# print(questions)

# Create a prompt with the template
prompt_tpl = PromptTemplate.from_template(tpl)
messages = [ HumanMessage(content=prompt_tpl.format(questions=questions)) ]
chat.invoke(messages)

resp = chat.batch([messages])

# get the list
titles = resp[0].content.split("\n")

# make sure the line count is the same
assert len(titles) == len(questions.split("\n"))

# now create a map for the short Titles to the long questions and write to a json file
shortTitles = {}
for i in range(len(titles)):
    shortTitles[titles[i]] = questions.split("\n")[i]

# write to a json file
import json
with open(data_folder + 'survey_data_titleMap.json', 'w') as outfile:
    json.dump(shortTitles, outfile)


# replace the titles in the df
df.columns = titles
df.head()

# write to a new csv with shortTitles
csv_survey_shortTitles = data_folder + 'survey_data_shortTitles.csv'  # Replace with desired output file path
df.to_csv(csv_survey_shortTitles, index=False)

